# DBNOMICS DATA WITH PYTHON#

Author: Andrea Sante
Last Modified: 05/10/2022

This small piece of code downloads and reads-in economic data from the DBnomics database. 
The program returns a csv file containing the data of interest.


In [1]:
#Install Python package 
%pip install jsonlines  
%pip install csv
%pip install requests

import requests
import jsonlines
import csv

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


Note: you may need to restart the kernel to use updated packages.


In [14]:
def select_series(JSON_VALUE):
    """
    Select series of interest ensuring they can be analysed properly.
    Return 'True' for the selected series.
    """
    keys_of_interest = ['series_id', 'name', 'units', 'lastHistoricalPeriod', 'data']
    boolean = False

    #Check that dictionary keys are in JSON file
    for key in keys_of_interest:
        if key in list(JSON_VALUE.keys()):
            boolean += True
    
    #If all the keys are in the dictionary
    if boolean==5:
        boolean = True
        
        #Select only data beginning with "REFERENCE." from 2021
        if "IEO.2021.REFERENCE" in JSON_VALUE['series_id']:
            boolean = True
        else:
            boolean = False
            
    #In case only some of the keys are present
    else:
        boolean = False
    
    return boolean       

def format_series_id(series_id):
    """
    Remove beginning of id name of the series (i.e. 'IEO.2021.REFERENCE.')
    """
    
    return series_id[19::]

def extract_values(series):
    """
    Separate values from periods in series
    """
    values = []
    
    for data_pair in series:
        values.append(data_pair[1])
    
    return list(reversed(values))

In [15]:
#Download dataset from url using requests library
url = 'https://git.nomics.world/dbnomics-source-data/eia-source-data/-/raw/master/IEO.txt'
r = requests.get(url, allow_redirects=True)

#Create a .jsonl file containing the downloaded data
open('dataset.jsonl', 'wb').write(r.content)

19058614

In [16]:
#The dataset is in a .jsonl format which contains one valid JSON value per line.

#Use jsonlines library to read in file
datafile = jsonlines.open('dataset.jsonl')

#Open .csv file where the data will be collected
file = open('output.csv', 'w', newline='')
writer = csv.writer(file)

#Write header
years = []
for i in range(2010,2051):
    years.append(str(i))
header = ['Series ID', 'Series Name', 'Units', 'Last Historical Update'] + years

writer.writerow(header)

#Iterate over each JSON value in the file.
for JSON_VALUE in datafile: 
    
    if select_series(JSON_VALUE):
        #Extract series id 
        series_id = format_series_id(JSON_VALUE['series_id'])
        #Extract series name
        series_name = JSON_VALUE['name']
        #Extract units
        series_units = JSON_VALUE['units']
        #Extract last historical update for the series
        lasthistoricalperiods = JSON_VALUE['lastHistoricalPeriod']
        #Extract data for the series 
        values = extract_values(JSON_VALUE['data'])
        
        writer.writerow([series_id, series_name, series_units, lasthistoricalperiods]+values)

file.close()